In [3]:
import pymongo
import pandas as pd

In [4]:
def db_conn(db_name):
    #connection to mongo database
    conn = pymongo.MongoClient('mongodb://200.18.98.107:27017')

    db_cpl = conn[db_name]
    
    return db_cpl

In [5]:
db_ap = db_conn('db_allpairs')
db_cpl = db_conn('db_cpl')

In [45]:
def promote_instance(category_name, limit, T):
    
    #find all promoted patterns
    category = (db_cpl.ontology
                .find_one({'category_name':category_name}))
                
    promoted_patterns = category['promoted_patterns']
    promoted_instances = category['promoted_instances']
    mutex_exceptions = category['mutex_exceptions']
    
    #get the promoted patterns of the mutex exception categories
    mutex_exception_query = (db_cpl.ontology.find({'category_name':{"$in":mutex_exceptions}}))   
    mutex_exception_patterns = [i['promoted_patterns'] for i in mutex_exception_query]
    mutex_exception_patterns = [item for sublist in mutex_exception_patterns for item in sublist]

    #count the ocurrences of all posive categories with each promoted pattern
    pipe_pos = [{"$match":{"$and":[{"ctx_pattern":{"$in":promoted_patterns}},       #extraction step
                                   {"noun_phrase":{"$nin":promoted_instances}}]}},  #filter step 
                {"$group": {"_id": "$noun_phrase", "count": {"$sum": "$counter"}}}, 
                {"$sort": {"count":-1}},                                            #rank step
                {"$limit": limit}]
    
    #count the ocurrences of all negative categories with each promoted pattern
    pipe_neg = [{"$match":{"$and":[{"ctx_pattern":{"$nin":promoted_patterns}},
                                   {"ctx_pattern":{"$nin":mutex_exception_patterns}}]}},
                {"$group": {"_id": "$noun_phrase", "count": {"$sum": "$counter"}}},
                {"$sort": {"count":-1}}]
    
    df_positive = pd.DataFrame(list(db_ap.allpairs.aggregate(pipe_pos))).set_index("_id")
    df_negative = pd.DataFrame(list(db_ap.allpairs.aggregate(pipe_neg))).set_index("_id")
    
    joined = (df_positive.join(df_negative, rsuffix='_neg')
              .fillna(0)
              .assign(tri_count_neg=lambda df: df['count'] >= df['count_neg']*T))  #promote step
    
    promoted_instances=list(joined[joined['tri_count_neg']].index.values)
    
    return promoted_instances

In [ ]:
def promote_ctx_pattern(category_name):
    #find all promoted patterns
    category = (db_cpl.ontology
                .find_one({'category_name':category_name}))

In [46]:
pr = promote_instance('city',10)

pr

             count
_id               
grapes         2.0
london         1.0
honda civic    1.0
mexico         1.0


['sao paulo', 'london', 'pittsburgh', 'berlin']

In [63]:
pipeline = [{"$match":{"ctx_pattern":{"$in":["live in _","is mayor of _"]}}},
            {"$group": {"_id": "$noun_phrase", "count": {"$sum": "$counter"}}}]

res = db.allpairs.aggregate(pipeline)

list(res)

[{'_id': 'berlin', 'count': 1.0},
 {'_id': 'pittsburgh', 'count': 3.0},
 {'_id': 'las vegas', 'count': 1.0},
 {'_id': 'sao paulo', 'count': 4.0},
 {'_id': 'london', 'count': 1.0}]

In [14]:
x = ['a','b','c']

y = **(x)

SyntaxError: invalid syntax (<ipython-input-14-51356296cb0d>, line 3)